In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# !pip uninstall scikit-image
# !pip install scikit-image
import numpy as np
import cv2
from skimage.feature import hog, local_binary_pattern
from sklearn.cluster import KMeans
from sklearn.svm import SVC
import pickle
import os
os.chdir("/content/gdrive/My Drive/")

In [0]:
def ReadData():
    data = []
    label = []
    for i in range(1, 6):
        with open("Colab Notebooks/Data/data_batch_" + str(i), 'rb') as fo:
            dict = pickle.load(fo, encoding='bytes')
            data.append(dict[b'data'])
            label.append(dict[b'labels'])

    for i in range(1, 5):
        data[0] = np.concatenate((data[0], data[i]), axis = 0)
        label[0] = np.concatenate((label[0], label[i]), axis = 0)

    test = []
    test_label = []
    with open("Colab Notebooks/Data/test_batch", 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
        test.append(dict[b'data'])
        test_label.append(dict[b'labels'])

    data = np.asarray(data[0])
    label = np.asarray(label[0])
    test = np.asarray(test[0])
    test_label = np.asarray(test_label[0])
    return data, label, test, test_label 

In [0]:
def HOG_features(x_train):
    fds = []
    for i in range(x_train.shape[0]):
        Image = np.reshape(x_train[i], (32, 32, 3))
        # cv2.imwrite("./Q4-output/original.png", Image)
        # fd, _ = hog(Image, orientations=8, pixels_per_cell=(4, 4), cells_per_block=(1, 1), visualize=True, multichannel=True, block_norm="L2-Hys")
        fd, _ = hog(Image, visualize=True, multichannel=True, block_norm="L2-Hys")

        fds.extend(fd)

    fds = np.asarray(fds)
    fds = np.reshape(fds, (fds.shape[0], 1))
    print("Fds.shape", fds.shape)
    return fds

def LBP_features(x_train, num_points = 24, radius = 3):
    fds = []
    for i in range(x_train.shape[0]):
        Image = np.reshape(x_train[i], (32, 32, 3))
        Image = cv2.cvtColor(Image, cv2.COLOR_BGR2GRAY)
        lbp = local_binary_pattern(Image, num_points, radius, method="uniform")
        # cv2.imwrite("./Q4-output/lbp1.png", lbp)
        hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, num_points + 3), range=(0, num_points + 2))
        hist = hist.astype(dtype=np.float)
        hist /= np.sum(hist)
        fds.extend(hist)
    
    fds = np.asarray(fds)
    fds = np.reshape(fds, (fds.shape[0], 1))
    print("Fds.shape", fds.shape)
    return fds

def getHistorgram(kmeans, x_data, num_points = 24, radius = 3, hog_flag=True):
    features = []
    hist_size = len(kmeans.cluster_centers_)
    # print("Clusters:", hist_size)
    for i in range(x_data.shape[0]):
        Image = np.reshape(x_data[i], (32, 32, 3))
        histogram = np.zeros((hist_size, ))
        if hog_flag:
            # fd, _ = hog(Image, orientations=8, pixels_per_cell=(4, 4), cells_per_block=(1, 1), visualize=True, multichannel=True, block_norm="L2-Hys")
            fd, _ = hog(Image, visualize=True, multichannel=True, block_norm="L2-Hys")
            norm = np.size(fd)
            fd = np.reshape(fd, (fd.shape[0], 1))
            histogram[kmeans.predict(fd)] += 1/norm
        else:
            Image = cv2.cvtColor(Image, cv2.COLOR_BGR2GRAY)
            lbp = local_binary_pattern(Image, num_points, radius, method="uniform")  # cv2.imwrite("./Q4-output/lbp1.png", lbp)
            hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, num_points + 3), range=(0, num_points + 2))
            hist = hist.astype(dtype=np.float)
            hist /= np.sum(hist)
            hist = np.reshape(hist, (hist.shape[0], 1))
            histogram[kmeans.predict(hist)] += 1
        features.append(histogram)
    features = np.asarray(features)
    return features

In [0]:
X_Train, Y_Train, X_Test, Y_Test = ReadData()
X_Train = X_Train[:10000]
Y_Train = Y_Train[:10000]
X_Test = X_Test[:1000]
Y_Test = Y_Test[:1000]


In [15]:
fd_list = HOG_features(X_Train)
# fd_list = LBP_features(X_Train)
hog_flag = True
print("------------------Features Collected-----------------")

Fds.shape (3240000, 1)
------------------Features Collected-----------------


In [16]:

kmeans = KMeans(n_clusters = 150)
kmeans.fit(fd_list)
print("------------------K-Mean Clutering Done--------------")

------------------K-Mean Clutering Done--------------


In [0]:
pickle.dump(kmeans, open("Colab Notebooks/Data/kmeans.sav", 'wb'))

In [18]:
nx_train = getHistorgram(kmeans, X_Train, hog_flag=hog_flag)
print("nx_train.shape", nx_train.shape)
nx_test = getHistorgram(kmeans, X_Test, hog_flag=hog_flag)
print("nx_test.shape", nx_test.shape)
print("------------------Histogram Features-----------------")

nx_train.shape (10000, 150)
nx_test.shape (1000, 150)
------------------Histogram Features-----------------


In [12]:
print("------------------Training SVM-----------------------")
clf = SVC(gamma='scale', decision_function_shape='ovo')
clf.fit(nx_train, Y_Train)

------------------Training SVM-----------------------


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [13]:
print("------------------Predicting Score-------------------")
Accuracy = clf.score(nx_test, Y_Test)

print("Acc", Accuracy*100)

------------------Predicting Score-------------------
Acc 19.6
